In [82]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from utils import plot_boxplot, get_seasonality_trend_overview, plot_period_mean, plot_periodogram, create_date_features, create_work_related_features, num_leading_zeros, num_trailing_zeros
import warnings
import statsmodels.api as sm
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [83]:
df_train = pd.read_csv('train_test_v4.csv', parse_dates=['date'])
df_train.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,local_holidays,regional_holidays,national_holidays
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1


In [84]:
df_train = create_date_features(df_train)
df_train.sample(5)

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,local_holidays,regional_holidays,national_holidays,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,date_index,season
379642,2013-08-02,3,EGGS,550.0,0.0,Quito,Pichincha,D,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,106.94,3213.0,0,0,0,8,2,214,1,31,5,2013,1,3,0,0,0,0,0,0,213,2
2895573,2017-06-13,49,LINGERIE,16.0,0.0,Quito,Pichincha,A,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.41,2609.0,0,0,0,6,13,164,2,24,2,2017,0,2,0,0,0,0,0,0,1624,2
2411146,2016-09-15,4,BABY CARE,0.0,0.0,Quito,Pichincha,D,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43.85,1197.0,0,0,0,9,15,259,3,37,4,2016,0,3,0,0,0,0,0,0,1353,3
422023,2013-08-25,45,LADIESWEAR,0.0,0.0,Quito,Pichincha,A,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,106.08,4138.0,0,0,0,8,25,237,4,34,7,2013,1,3,0,0,0,0,0,0,236,2
2548993,2016-12-01,23,CLEANING,491.0,6.0,Ambato,Tungurahua,D,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.08,921.0,0,0,0,12,1,336,1,48,4,2016,0,4,1,0,0,0,0,0,1430,0


In [85]:
train_wide = pd.pivot(df_train.groupby(['date','store_nbr'])['sales'].agg('sum').reset_index(), index=['date'], columns=['store_nbr'], values=['sales'])
train_wide.head()

sales                                            \
store_nbr            1             2             3             4    
date                                                                
2013-01-01     0.000000      0.000000      0.000000      0.000000   
2013-01-02  7417.148000  10266.718981  24060.348000  10200.083980   
2013-01-03  5873.244001   7612.540000  18570.745025   7400.245000   
2013-01-04  5919.879001   7526.377000  17392.097995   6902.050001   
2013-01-05  6318.785010   9539.566000  22700.872005   9948.383000   

                                                                              \
store_nbr             5             6             7            8          9    
date                                                                           
2013-01-01      0.000000      0.000000      0.000000      0.00000      0.000   
2013-01-02  10598.619020  13520.485002  11997.502000  14659.32800  15867.484   
2013-01-03   7770.968000   9929.393000   9277.958000  10781.24300  10431.719   
2013-01-04   7347.641991   8869.211980   8096.573018  11124.65100   8085.159   
2013-01-05   8192.598000  12431.689040   8610.605999  14277.69902  13057.110   

                                                                             \
store_nbr           10            11           12           13           14   
date                                                                          
2013-01-01     0.00000      0.000000     0.000000     0.000000     0.000000   
2013-01-02  5430.08800  18329.638000  5854.517000  5516.831000  7698.253000   
2013-01-03  4549.52998  12356.205040  4460.164000  4852.438995  6158.575996   
2013-01-04  3381.01600  12171.998022  3684.053004  4249.484000  5777.689000   
2013-01-05  5409.78501  14763.049000  4818.668000  6465.232000  7974.621000   

                                                                           \
store_nbr           15        16           17        18           19   20   
date                                                                        
2013-01-01     0.00000     0.000     0.000000     0.000     0.000000  0.0   
2013-01-02  6577.80500  6242.572  7673.513000  8204.567  5615.389008  0.0   
2013-01-03  5691.44700  5316.415  5523.241000  5107.041  3783.087000  0.0   
2013-01-04  4825.64699  4616.990  5239.568995  4033.748  3233.503000  0.0   
2013-01-05  5694.05600  6014.655  7557.546000  6510.638  6363.821023  0.0   

                                                                       \
store_nbr    21   22          23            24           25        26   
date                                                                    
2013-01-01  0.0  0.0     0.00000      0.000000  2511.618999     0.000   
2013-01-02  0.0  0.0  6560.33700  11018.852996  5316.224000  5653.413   
2013-01-03  0.0  0.0  4911.20401  10086.240998  4442.913020  3477.559   
2013-01-04  0.0  0.0  5620.52800  10059.480998  4844.354000  3538.449   
2013-01-05  0.0  0.0  6041.00200  12031.909020  5817.526005  5503.186   

                                                                    \
store_nbr            27           28   29           30          31   
date                                                                 
2013-01-01     0.000000     0.000000  0.0     0.000000     0.00000   
2013-01-02  7207.547000  5176.474000  0.0  2623.009000  6637.56000   
2013-01-03  4387.566000  4142.304000  0.0  2538.315003  5437.82600   
2013-01-04  4586.466001  2615.644994  0.0  1987.330000  6107.63600   
2013-01-05  6731.607015  5939.632000  0.0  2811.187998  5875.18901   

                                                                     \
store_nbr            32           33           34           35   36   
date                                                                  
2013-01-01     0.000000     0.000000     0.000000     0.000000  0.0   
2013-01-02  2441.508000  5720.770998  7749.693000  2978.224000  0.0   
2013-01-03  2589.699001  5167.402000  6246.722000  2699.277000  0.0   
20

Trailing zeros are only in test set so we dont use them as new attributes

In [86]:
leading_zeros = train_wide.apply(num_leading_zeros).to_frame('num_leading_zeros').reset_index()
leading_zeros = leading_zeros[leading_zeros['num_leading_zeros'] != train_wide.shape[0]]
leading_zeros

,level_0,store_nbr,num_leading_zeros
0,sales,1,1
1,sales,2,1
2,sales,3,1
3,sales,4,1
4,sales,5,1
5,sales,6,1
6,sales,7,1
7,sales,8,1
8,sales,9,1
9,sales,10,1


In [87]:
from datetime import datetime, timedelta

def check_for_leading(df: pd.Series):
    # if less then its not workday
    days_add = leading_zeros[leading_zeros['store_nbr'] == df['store_nbr']]['num_leading_zeros']
    if df['date'] < datetime.strptime('01-01-2013', '%d-%m-%Y') + timedelta(days=int(days_add)):
        return 0
    else:
        return 1

df_train['workday_because_of_leading_zeroes'] = df_train.apply(check_for_leading, axis=1)

In [88]:
df_train.sample(7)

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,local_holidays,regional_holidays,national_holidays,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,date_index,season,workday_because_of_leading_zeroes
2777386,2017-04-08,32,CLEANING,655.0,15.0,Guayaquil,Guayas,C,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,52.520000,705.0,0,0,0,4,8,98,2,14,6,2017,1,2,0,0,0,0,0,0,1558,1,1
777357,2014-03-13,13,DELI,103.0,0.0,Latacunga,Cotopaxi,C,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,98.570000,791.0,0,0,0,3,13,72,2,11,4,2014,0,1,0,0,0,0,0,0,436,1,1
1338566,2015-01-22,9,LAWN AND GARDEN,7.0,0.0,Quito,Pichincha,B,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45.930000,1923.0,0,0,0,1,22,22,4,4,4,2015,0,1,0,0,0,0,0,0,751,0,1
1263325,2014-12-10,51,LADIESWEAR,7.0,0.0,Guayaquil,Guayas,A,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,60.990000,2161.0,0,0,0,12,10,344,2,50,3,2014,0,4,0,0,0,0,0,0,708,0,1
228918,2013-05-09,25,PRODUCE,0.0,0.0,Salinas,Santa Elena,D,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96.090000,658.0,0,0,0,5,9,129,2,19,4,2013,0,2,0,0,0,0,0,0,128,1,1
1419711,2015-03-08,38,HOME CARE,0.0,0.0,Loja,Loja,D,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,49.836667,2639.0,0,0,0,3,8,67,2,10,7,2015,1,1,0,0,0,0,0,0,796,1,1
313522,2013-06-25,51,"LIQUOR,WINE,BEER",55.0,0.0,Guayaquil,Guayas,A,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,95.250000,1504.0,0,0,0,6,25,176,4,26,2,2013,0,2,0,0,0,0,0,0,175,2,1


In [89]:
def create_work_related_features(df: pd.DataFrame):
    df['workday'] = np.where((df.local_holidays == 1) | (df.national_holidays == 1) | (df.regional_holidays == 1) | (df['day_of_week'].isin([6,7])) | (df.workday_because_of_leading_zeroes == 0), 0, 1)
    df['wageday'] = pd.Series(np.where((df['is_month_end'] == 1) | (df['day_of_month'] == 15), 1, 0)).astype('int8')
    df.drop(columns='workday_because_of_leading_zeroes', inplace=True)
    return df

df_train = create_work_related_features(df_train)

In [91]:
df_train.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,local_holidays,regional_holidays,national_holidays,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,date_index,season,workday,wageday
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0


In [179]:
%pip install cacheout
from cacheout import Cache

cache = Cache()

@cache.memoize()
def days_to_holiday(date, holidays, from_holiday=True):
    return (date['date'] - min([h for h in holidays if h <= date['date']], key=lambda x: abs((date['date'] - x).days))).days if from_holiday else (min([h for h in holidays if h >= date['date']], key=lambda x: abs((x - date['date']).days)) - date['date']).days


def create_holiday_features(df, df_holidays):
    df['day_to_nearest_holiday'] = df.apply(lambda x: days_to_holiday(x, df_holidays), axis=1)
    df['day_from_nearest_holiday'] = df.apply(lambda x: days_to_holiday(x, df_holidays,  from_holiday=False), axis=1)
    return df  

df_holidays = pd.read_csv('originalni_datasetovi/holidays_events.csv', parse_dates=['date'])
df_holidays = list(set(df_holidays.loc[df_holidays['locale'] == 'National']['date'].to_list()))
df_holidays.sort()
df_traini = create_holiday_features(df_train, df_holidays)
df_traini.head()

Note: you may need to restart the kernel to use updated packages.


,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,local_holidays,regional_holidays,national_holidays,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,date_index,season,workday,wageday,day_to_nearest_holiday,day_from_nearest_holiday
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0


In [180]:
df_traini.to_csv('posle_holidaya.csv')